In [1]:
from datasets import load_dataset
import pandas as pd
import sys
from datasets import Dataset
from sentence_transformers.util import mine_hard_negatives
from sentence_transformers import SentenceTransformer

sys.path.append("..")

In [2]:
pd.set_option('display.max_colwidth', None)  # None means no limit on column width
pd.set_option('display.width', 300)  

In [3]:
ds_alpaca_srb = load_dataset("datatab/alpaca-cleaned-serbian-full")
ds_alpaca_srb

DatasetDict({
    train: Dataset({
        features: ['input', 'instruction', 'output'],
        num_rows: 51760
    })
})

In [4]:
ds_alplaca_eng = load_dataset("yahma/alpaca-cleaned")
ds_alplaca_eng

DatasetDict({
    train: Dataset({
        features: ['output', 'input', 'instruction'],
        num_rows: 51760
    })
})

In [6]:
# input_user = ds_alpaca['train']['input']
instruction_srb = ds_alpaca_srb['train']['instruction']
output_srb = ds_alpaca_srb['train']['output']

instruction_eng = ds_alplaca_eng['train']['instruction']
output_eng = ds_alplaca_eng['train']['output']

df = pd.DataFrame({
                #    "input" : input_user,
                   "input_srb" : instruction_srb,
                   "output_srb" : output_srb,
                   "input_engl": instruction_eng,
                   "output_eng": output_eng})

In [8]:
df.shape

(51760, 4)

In [16]:
df.sample(20)

,input_srb,output_srb,input_engl,output_eng
45260,Nađi površinu ovog pravougaonika: 5 cm k 2 cm,"Površina pravougaonika se izračunava množenjem dužine sa širinom. U ovom slučaju, dužina je 5 cm, a širina 2 cm. Dakle, površina pravougaonika je (5 cm) * (2 cm) = 10 cm².",Find the area of this rectangle: 5 cm x 2 cm,"The area of a rectangle is calculated by multiplying the length by the width. In this case, the length is 5 cm and the width is 2 cm. So, the area of the rectangle is (5 cm) * (2 cm) = 10 cm²."
44265,"Sastavite i postavite tvit o filmu ""Titanik"".","""Doživite epsku romansu i tragediju na #Titaniku. Ovaj klasični film iz 1997. odvešće vas na nezaboravno putovanje ljubavi i gubitka. #film #Džejms Kameron""\n\n***Imajte na umu da, kao AI, nemam mogućnost objavljivanja na društvenim mrežama, tako da ne mogu da sprečim (objavim) tvit.","Compose and belay a tweet about the film ""Titanic"".","""Experience the epic romance and tragedy aboard the #Titanic. This 1997 classic film will take you on an unforgettable journey of love and loss. #movie #JamesCameron"" \n\n***Please Note that, As an AI, I don't have the ability to post on social media so I am unable to belay (post) the tweet."
31789,Navedite tri strategije za negovanje kreativnosti,"1. Divergentno razmišljanje: Ovo uključuje aktivno traženje novih iskustava, ideja i perspektiva, i izazivanje konvencionalnih načina razmišljanja. To znači prevazići dostupne informacije ili tradicionalne pristupe rešavanju problema i tražiti više odgovora ili rešenja za problem umesto samo jednog.\n\n2. Eksperimentisanje i preuzimanje rizika: Kreativnost često uključuje preuzimanje rizika i eksperimentisanje sa novim načinima rada. To može značiti isprobavanje nove tehnike, istraživanje novog hobija ili osporavanje sopstvenih uverenja i ideja. Izlaskom iz svojih zona udobnosti, pojedinci mogu naučiti da prevaziđu svoje strahove i otkriju nove mogućnosti.\n\n3. Radoznalost i otvorenost: radoznalost je ključni element u podsticanju kreativnosti. Istražujući svet oko sebe i postavljajući pitanja, negujemo svoju radoznalost, maštu i znanje. Otvorenost za nova iskustva i ideje, čak i ako izgledaju čudno ili nekonvencionalno, može nam pomoći da razvijemo naše kreativno razmišljanje i veštine rešavanja problema.",Name three strategies for cultivating creativity,"1. Divergent thinking: This involves actively seeking out new experiences, ideas, and perspectives, and challenging conventional ways of thinking. It means going beyond the available information or the traditional approaches to problem-solving and seeking for multiple answers or solutions to a problem instead of just one.\n\n2. Experimentation and taking risks: Creativity often involves taking risks and experimenting with new ways of doing things. This might mean trying a new technique, exploring a new hobby, or challenging one's own beliefs and ideas. By stepping out of their comfort zones, individuals can learn to overcome their fears and discover new possibilities.\n\n3. Curiosity and open-mindedness: Curiosity is a crucial element in fostering creativity. By exploring the world around us and asking questions, we nurture our curiosity, imagination, and knowledge. Being open-minded to new experiences and ideas, even if they may seem strange or unconventional, can help us develop our creative thinking and problem-solving skills."
45424,Opiši karakteristike date vrste cveća,"Ruže su cvetne vrste koje pripadaju porodici Rosaceae koja se može pohvaliti preko stotinu vrsta i hiljadama sorti. Nalaze se u širokom spektru boja, uključujući crvenu, belu, žutu, ružičastu, narandžastu, pa čak i zelenu ili crnu. Cvetovi se razlikuju po veličini, a neki su široki samo nekoliko inča, dok su drugi mnogo veći.\n\nRuže su cenjene zbog svoje lepote i mirisa. Cvetovi su često u obliku čaše sa preklapajućim slojevima mekih latica koje mogu biti prilično debele, dajući cvetovima pun, bujan izgled. Neke sorte imaju latice koje su baršunaste na dodir, do

In [10]:
df_parallel = df[['input_engl', 'input_srb']]
df_parallel.sample(15)

,input_engl,input_srb
40891,Create a poem in 123 words,Napravite pesmu od 123 reči
16208,Write a formal email to the boss politely asking for leave the next day.,Napišite službenu e-poruku šefu u kojoj ljubazno tražite odsustvo sledećeg dana.
51629,Outline the essential elements of a good argument.,Navedite osnovne elemente dobrog argumenta.
45942,Name and describe four types of renewable energy.,Navedite i opišite četiri vrste obnovljive energije.
38553,Compare and contrast the effectiveness of two popular search engines.,Uporedite i uporedite efikasnost dva popularna pretraživača.
8909,Generate a list of 10 questions about GPT models.,Napravite listu od 10 pitanja o GPT modelima.
10675,Automate an email response confirming a user's registration,Automatizujte odgovor e-poštom koji potvrđuje registraciju korisnika
13252,Calculate the speed of light in terms or Mach,Izračunajte brzinu svetlosti u terminima ili mahovima
41581,Create an example of a song title with a figurative meaning.,Napravite primer naslova pesme sa figurativnim značenjem.
45016,"Summarize the main ideas of the paper: ""The impact of AI on healthcare""",Sumirajte glavne ideje rada: „Uticaj AI na zdravstvenu zaštitu“


In [11]:
df_parallel['input_engl'] = df_parallel['input_engl'].str.replace('\n', ' ', regex=False)
df_parallel['input_srb'] = df_parallel['input_srb'].str.replace('\n', ' ', regex=False)
df_parallel.head()

/tmp/ipykernel_6168/1784720640.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_parallel['input_engl'] = df_parallel['input_engl'].str.replace('\n', ' ', regex=False)
/tmp/ipykernel_6168/1784720640.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_parallel['input_srb'] = df_parallel['input_srb'].str.replace('\n', ' ', regex=False)


,input_engl,input_srb
0,Give three tips for staying healthy.,Dajte tri saveta kako da ostanete zdravi.
1,What are the three primary colors?,Koje su tri osnovne boje?
2,Describe the structure of an atom.,Opišite strukturu atoma.
3,How can we reduce air pollution?,Kako možemo smanjiti zagađenje vazduha?
4,Pretend you are a project manager of a construction company. Describe a time when you had to make a difficult decision.,Pretvarajte se da ste menadžer projekta građevinske kompanije. Opišite trenutak kada ste morali da donesete tešku odluku.


In [12]:
df_parallel_sample = df_parallel.sample(n=10000, random_state=42)

In [13]:
def convert_to_hf_dataset(dataframe: pd.DataFrame) -> Dataset:
    # Convert each InputExample into a dictionary
    data_dict = {
        "anchor": [],
        "positive": [],
    }
    for _, row in dataframe.iterrows():
        data_dict['anchor'].append(row['input_engl'])
        data_dict['positive'].append(row['input_srb'])
    # Create a Hugging Face Dataset
    return Dataset.from_dict(data_dict)

In [14]:
ds_parallel = convert_to_hf_dataset(df_parallel_sample)
ds_parallel

Dataset({
    features: ['anchor', 'positive'],
    num_rows: 10000
})

In [15]:
model = SentenceTransformer("intfloat/multilingual-e5-base")


In [ ]:
# dataset_neg = mine_hard_negatives(
#     dataset=ds_parallel,
#     model=model,
#     range_min=10,
#     range_max=100,
#     max_score=0.9,
#     margin=0.01,
#     num_negatives=2,
#     sampling_strategy="random",
#     batch_size=128,
#     use_faiss=True,
# )

In [ ]:
# df_neg = pd.DataFrame(dataset_neg)
# df_neg.sample(10)

In [ ]:
# positive_examples = df_neg[['anchor', 'positive']].copy()
# positive_examples.columns = ['sentence1', 'sentence2']
# positive_examples['label'] = 1

# negative_examples = df_neg[['anchor', 'negative']].copy()
# negative_examples.columns = ['sentence1', 'sentence2']
# negative_examples['label'] = 0

# df_transformed = pd.concat([positive_examples, negative_examples], ignore_index=True).sort_values(by='sentence1')

In [ ]:
# df_transformed.drop_duplicates(subset = 'sentence2', keep='first', inplace=True, ignore_index=True)

In [ ]:
# df_transformed.to_parquet("../datasets/sr_en_pos_neg_with_label.parquet")

In [ ]:
# df_neg.to_parquet("../datasets/sr_en_pos_neg_triplets.parquet")

In [ ]:
# df_parallel_sample.to_parquet("../datasets/sr_en_parallel_10k.parquet")

In [ ]:
ds_orca['train']['conversations'][0]

In [ ]:
human = next(item['value'] for item in ds_orca['train']['conversations'][0] if item['from'] == 'human')
gpt = next(item['value'] for item in ds_orca['train']['conversations'][0] if item['from'] == 'gpt')

print("Human:", human)
print("GPT:", gpt)

In [18]:
tatoeba = load_dataset("tatoeba", lang1="en", lang2="sr")
tatoeba

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 21300
    })
})

In [19]:
idx = tatoeba['train']['id']
en = [tatoeba['train']['translation'][i]['en'] for i in tatoeba['train']['translation']]

TypeError: list indices must be integers or slices, not dict

In [20]:
tatoeba['train'
]['translation']

[{'en': "Let's try something.", 'sr': 'Hajde da probamo nešto.'},
 {'en': 'I have to go to sleep.', 'sr': 'Морам да идем да спавам.'},
 {'en': "Today is June 18th and it is Muiriel's birthday!",
  'sr': 'Danas je 18. Juni i Mjurijelov je rođendan.'},
 {'en': 'Muiriel is 20 now.', 'sr': 'Mjurijelu je sada 20.'},
 {'en': 'The password is "Muiriel".', 'sr': 'Шифра је "Мјуриел".'},
 {'en': 'The password is "Muiriel".', 'sr': 'Šifra je "Mjurijel".'},
 {'en': 'I will be back soon.', 'sr': 'Vraćam se uskoro.'},
 {'en': 'This is never going to end.', 'sr': 'Ovo se nikada neće završiti.'},
 {'en': "I just don't know what to say.",
  'sr': 'Prosto ne znam šta da kažem.'},
 {'en': 'That was an evil bunny.', 'sr': 'To je bio zli zec.'},
 {'en': 'I was in the mountains.', 'sr': 'Bio sam u planinama.'},
 {'en': "I don't know if I have the time.",
  'sr': 'Не знам да ли ћу имати времена.'},
 {'en': 'Education in this world disappoints me.',
  'sr': 'U našem svetu obrazovanje je razočaravajuće.'},
 {'

In [ ]:
data = pd.read_csv('../datasets/STS.news.sr.txt', sep="\t", header=None)
data.columns = ["AvgScore", "Score1", "Score2", "Score3", "Score4", "Score5", "Sentence1", "Sentence2"]

In [23]:
wikimatrix = pd.read_parquet("../datasets/train.parquet")
wikimatrix.shape

(20000, 5)

In [25]:
wikimatrix.sample(15)

,lang1,sentence2,guid,lang2,sentence1
4802,en,"Породица Трентон, која припада средњој класи, недавно се преселила у Кесл Рок из Њујорка, са собом повевши четворогодишњег сина Теда.",3321,sr,"The middle-class Trentons have recently moved to Castle Rock from New York City, bringing with them their four-year-old son, Tad."
5382,en,"У јавном мњењу 1998. године, 78,3% Бошњака у Федерацији Босне и Херцеговине се изјаснило да су верници.",18785,sr,"In a 1998 public opinion poll, 78.3% of Bosniaks in the Federation of Bosnia and Herzegovina declared themselves to be religious."
4513,en,Одбијено му је продужење визе од стране режима Мугабе због свог политичког ангажмана са покретом за демократију.,4422,sr,He was refused a visa extension by the Mugabe regime for his political involvement with the pro-democracy movement.
18269,en,Знак да неко има астрафобију је веома изражен интерес за временску прогнозу.,22197,sr,A sign that someone has astraphobia is a very heightened interest in weather forecasts.
1428,en,"Власништво над храмом и земљиштем на којем се простире, налази се у спору између садашњег Универзитета у Приштини и Српске православне цркве.",12982,sr,Ownership of the building and the land on which it is located is disputed between the current University of Pristina and the Serbian Orthodox Church.
16852,en,Булгаков је касније написао представу на основу приче у 1926. години за Московско уметничко позориште.,5659,sr,Bulgakov subsequently wrote a play based on the story in 1926 for the Moscow Art Theater.
12722,en,Милтон је такође уобличио главне теме будућих дискусија о слободи изражавања.,1368,sr,But Milton also articulated the main strands of future discussions about freedom of expression.
18198,en,"Ovo je bio bitan koncept, mada se njihovo sredstvo za automatsku analizu pokazalo potpuno razočaravajućim.",17066,sr,"This was an important concept, though their automated analysis tool proved ultimately disappointing."
13365,en,Сад се вратио сам у Медину; Умајр је био један од четрнаест муслимана који су погинули у бици.,20282,sr,Saʿd returned to Medina alone; ʿUmayr was one of the fourteen Muslims who died in the battle.
212,en,"Ovi članovi proširene porodice se često okupljaju radi porodičnih događaja i kako bi pomogli i podržali jedni druge, i emotivno i finansijski.",5351,sr,"These extended family members tend to gather often for family events and to feel responsible for helping and supporting one another, both emotionally and financially."


In [26]:
wikimatrix_fixed = pd.DataFrame({"lang1" : wikimatrix['lang1'],
                                 "sentence1" : wikimatrix['sentence1'],
                                 "lang2" : wikimatrix['lang2'],
                                 "sentence2" : wikimatrix['sentence2']})

In [ ]:
wikimatrix_fixed.sample(30)

,lang1,sentence1,lang2,sentence2
10779,en,"When the Museum of the Chinese Revolution opened on Tiananmen Square in 1961, the painting was displayed on a huge wall in the gallery devoted to the Communist triumph, but in 1966, during the Cultural Revolution, radicals shut down the museum, and it remained closed until 1969.",sr,"Када је 1961. године на Тргу Тјенанмен отворен Музеј кинеске револуције, слика је била изложена на огромном зиду у галерији посвећеној комунистичком тријумфу, али 1966. године, током Културне револуције, радикали су затворили музеј, и остао је затворен до 1969."
15885,en,"The scholars also inspired in the King an interest in Judaism, and he converted in 390, persuading his army to do likewise.",sr,"Научници су такође надахнули код краља интересовање за јудаизм, а он је прешао на јудаизам 390. године, и убедио своју војску да учини исто."
10344,en,"Chivas Regal 25 Year Old was launched in 1909 as the original luxury Scotch, and became a leading brand in the United States.",sr,"Чивас Регал стар 25 година је пуштен у продају 1909. као луксузни оригинални Шкотски виски, и постао је водећи бренд у Сједињеним Државама."
17969,en,Elizabeth visits the ill Jane at Netherfield.,sr,Elizabet posećuje bolesnu Džejn u Nederfildu.
4468,en,"Ancient Greek legends told of sea voyages to Cilicia, where adventurers sought what they believed were the world's most valuable threads.",sr,"Древне грчке легенде говоре о морским путовањима до Киликије, где су авантуристи тражили оно за шта су веровали да су највреднији кончићи на свету."
4272,en,"The quantities of supplies and intelligence from the captured documents revealed that at least 3,000 Japanese troops were on the island and apparently planning an attack.",sr,"Према количини залиха и обавештења из заробљених докумената, откривено је да се најмање 3.000 јапанских војника налазило на острву, и очигледно планирају напад."
14321,en,"Law is philosophy, because moral and ethical persuasions shape their ideas.",sr,"Право је филозофија, јер морална и етичка уверења формирају идеје."
7864,en,The positive-real (PR) condition is both necessary and sufficient but there may be practical reasons for rejecting some topologies.,sr,Pozitivno-realni uslov (PR) je i neophodan i dovoljan ali može biti praktičnih razloga za odbacivanje nekih topologija. topologies.
16751,en,"Fernand Braudel: who had described the development and political implications of extensive networks of economic exchange in the European world between 1400 and 1800, ""more than anyone else made me conscious of the central importance of the social construction of time and space and its impact on our analyses.""",sr,"Фернан Бродел: који је описао развој и политичка уплитања разгранатих мрежа економских размена у европском свету између 1400. и 1800, ""више од било кога ме је учинио свесним примарног значаја социјалне изградње простора и времена и њихов утицај на наше анализе."""
9119,en,Sudan has reportedly also expressed an interest in acquiring of the Su-35 fighters during the Sudanese president Omar Hassan al-Bashir's visitation of Moscow in November 2017.,sr,"Судан је, како се извештава, такође изразио интересовање за куповину ловца Су-35, током посете суданског Председника Омар Хасана ал-Башира Москви у новембру 2017. године."


In [34]:
wikimatrix_fixed.to_parquet("../datasets/wikimatrix.parquet")

In [33]:
pd.set_option('display.max_colwidth', None)  # None means no limit on column width
pd.set_option('display.width', 300)  

In [35]:
sts = load_dataset("mteb/stsbenchmark-sts")
sts

Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1379 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['split', 'genre', 'dataset', 'year', 'sid', 'score', 'sentence1', 'sentence2'],
        num_rows: 5749
    })
    validation: Dataset({
        features: ['split', 'genre', 'dataset', 'year', 'sid', 'score', 'sentence1', 'sentence2'],
        num_rows: 1500
    })
    test: Dataset({
        features: ['split', 'genre', 'dataset', 'year', 'sid', 'score', 'sentence1', 'sentence2'],
        num_rows: 1379
    })
})

In [36]:
sts['test']

Dataset({
    features: ['split', 'genre', 'dataset', 'year', 'sid', 'score', 'sentence1', 'sentence2'],
    num_rows: 1379
})

In [40]:
sts_test_df = pd.DataFrame(sts['test'])
sts_test_df.head()

,split,genre,dataset,year,sid,score,sentence1,sentence2
0,test,main-captions,MSRvid,2012test,0024,2.5,A girl is styling her hair.,A girl is brushing her hair.
1,test,main-captions,MSRvid,2012test,0033,3.6,A group of men play soccer on the beach.,A group of boys are playing soccer on the beach.
2,test,main-captions,MSRvid,2012test,0045,5.0,One woman is measuring another woman's ankle.,A woman measures another woman's ankle.
3,test,main-captions,MSRvid,2012test,0063,4.2,A man is cutting up a cucumber.,A man is slicing a cucumber.
4,test,main-captions,MSRvid,2012test,0066,1.5,A man is playing a harp.,A man is playing a keyboard.


In [41]:
sts_test_df.sample(10)

,split,genre,dataset,year,sid,score,sentence1,sentence2
506,test,main-captions,track5.en-en,2017,0014,2.80,A man is performing labor.,A man is performing today.
43,test,main-captions,MSRvid,2012test,0281,0.50,A man is playing a guitar.,A man is driving a car.
968,test,main-news,MSRpar,2012test,0561,3.00,"Chaudhry was released 56 days later and the Fiji military installed an all-indigenous government led by Qarase, who won democratic elections in September 2001.","The Fijian military installed an all-indigenous government led by Mr Qarase, who won democratic elections in September 2001."
1201,test,main-news,headlines,2014,0001,0.40,Opposition leaders emerge to commemorate Cambodian workers' deaths,Cambodia opposition leaders summoned to court
188,test,main-captions,MSRvid,2012train,0392,0.75,A man is making a bed.,A man is speaking to an audience.
988,test,main-news,MSRpar,2012test,0636,3.40,The Cradle of Liberty Council isn't the first one to buck the national group's stance.,Philadelphia's council is not the first to defy the national policy.
64,test,main-captions,MSRvid,2012test,0400,3.40,A person is playing an electronic keyboard.,A kid is playing keyboard.
316,test,main-captions,images,2014,0390,3.00,Three people are by the water with yellow canoes.,Three people preparing to canoe.
908,test,main-news,MSRpar,2012test,0150,3.00,"In 1999, the building's owners, the Port Authority of New York and New Jersey, issued guidelines to upgrade the fireproofing to a thickness of 1{ inches.",The NIST discovered that in 1999 the Port Authority issued guidelines to upgrade the fireproofing to a thickness of 1 1/2 inches.
695,test,main-forums,answer-answer,2016,0411,0.00,The best thing you can do is to know your stuff.,"My recommendation is not to say anything, and do the best you can."


In [43]:
sts_test_df_final = sts_test_df[["sid", 'sentence1', 'sentence2']]

In [44]:
sts_test_df_final

,sid,sentence1,sentence2
0,0024,A girl is styling her hair.,A girl is brushing her hair.
1,0033,A group of men play soccer on the beach.,A group of boys are playing soccer on the beach.
2,0045,One woman is measuring another woman's ankle.,A woman measures another woman's ankle.
3,0063,A man is cutting up a cucumber.,A man is slicing a cucumber.
4,0066,A man is playing a harp.,A man is playing a keyboard.
...,...,...,...
1374,1354,"Philippines, Canada pledge to further boost relations",Philippines saves 100 after ferry sinks
1375,1360,Israel bars Palestinians from Jerusalem's Old City,"Two-state solution between Palestinians, Israel pie in sky"
1376,1368,How much do you know about Secret Service?,Lawmakers from both sides express outrage at Secret Service
1377,1420,Obama Struggles to Soothe Saudi Fears As Iran Talks Resume,Myanmar Struggles to Finalize Voter Lists for Sunday Polls


In [45]:
type(sts_test_df['sid'][0])

str